# FastAPI - Virtual Environments (uv-focused)

**Source:** [FastAPI Docs - Virtual Environments](https://fastapi.tiangolo.com/virtual-environments/)

Practical command reference for virtual environments with `uv`.

---

## Core Concepts (Quick Reminder)

**Virtual Environment** = Isolated Python environment per project  
**Problem** = Different projects need different package versions  
**Solution** = Each project has its own `.venv/` directory with packages  
**Activation** = Prepends `.venv/bin` to `PATH` so `python` finds the right interpreter

## Project Setup Workflow

### Standard venv Method

```bash
# Create project
mkdir awesome-project && cd awesome-project

# Create venv
python -m venv .venv

# Activate (Linux/macOS)
source .venv/bin/activate

# Activate (Windows PowerShell)
.venv\Scripts\Activate.ps1

# Activate (Windows Bash/Git Bash)
source .venv/Scripts/activate

# Deactivate
deactivate
```

### uv Method (Recommended)

```bash
# Create project
mkdir awesome-project && cd awesome-project

# Create venv (creates .venv by default)
uv venv

# Create venv with custom name
uv venv myenv

# Create venv with specific Python version
uv venv --python 3.12
uv venv --python python3.11

# Activate (same as standard method)
source .venv/bin/activate  # Linux/macOS
.venv\Scripts\Activate.ps1  # Windows PowerShell

# Deactivate
deactivate
```

**uv Bonus:** Automatically creates `.gitignore` in `.venv/`

## Package Management with uv

### Installing Packages

```bash
# Must activate venv first!
source .venv/bin/activate

# Install single package
uv pip install fastapi

# Install with extras
uv pip install "fastapi[standard]"

# Install specific version
uv pip install "fastapi==0.115.0"

# Install from requirements.txt
uv pip install -r requirements.txt

# Install from pyproject.toml (if [project] dependencies defined)
uv pip install -e .

# Upgrade package
uv pip install --upgrade fastapi

# Install multiple packages
uv pip install fastapi uvicorn sqlalchemy
```

### Alternative: Using `uv add` (Project Management)

**Important:** `uv add` is for projects managed by uv with `pyproject.toml`

```bash
# Initialize uv project (creates pyproject.toml)
uv init

# Add package (updates pyproject.toml + installs)
uv add fastapi

# Add with extras
uv add "fastapi[standard]"

# Add dev dependency
uv add --dev pytest

# Add with version constraint
uv add "fastapi>=0.115.0"

# Sync dependencies (install from pyproject.toml)
uv sync

# Remove package
uv remove fastapi
```

## Key Difference: `uv pip install` vs `uv add`

| Command | Purpose | Updates pyproject.toml | Use Case |
|---------|---------|----------------------|----------|
| `uv pip install` | Install only | ❌ No | Manual venv management, requirements.txt |
| `uv add` | Install + track | ✅ Yes | uv-managed projects with pyproject.toml |

**Choose:**
- `uv pip install` when using traditional requirements.txt workflow
- `uv add` when using uv's full project management

## Checking and Listing

```bash
# Check which Python is active (should show .venv path)
which python        # Linux/macOS
Get-Command python  # Windows PowerShell

# Check Python version
python --version

# List installed packages
uv pip list

# Show package details
uv pip show fastapi

# Freeze dependencies (for requirements.txt)
uv pip freeze > requirements.txt

# Check outdated packages
uv pip list --outdated
```

## The PATH Mechanism (What Activation Does)

### Before Activation

```bash
# PATH (Linux/macOS)
/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin

# PATH (Windows)
C:\Windows\System32;C:\Program Files\Python312
```

### After Activation

```bash
# PATH (Linux/macOS) - .venv/bin PREPENDED
/home/user/project/.venv/bin:/usr/local/bin:/usr/bin:/bin

# PATH (Windows) - .venv\Scripts PREPENDED
C:\Users\user\project\.venv\Scripts;C:\Windows\System32
```

**Key Point:** `.venv/bin` is added at the **beginning**, so it's searched **first**

When you type `python`, the system finds `.venv/bin/python` before any global Python.

## Verify Active Environment

In [ ]:
import sys
import os

# Show Python executable path
print(f"Python executable: {sys.executable}")

# Check if in virtual environment
in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
print(f"In virtual environment: {in_venv}")

# Show sys.prefix (venv location)
print(f"sys.prefix: {sys.prefix}")

# Show PATH
path = os.environ.get('PATH', '')
print(f"\nPATH entries:")
separator = ';' if os.name == 'nt' else ':'
for p in path.split(separator)[:5]:  # Show first 5
    print(f"  {p}")

## uv Edge Cases and Gotchas

### 1. uv Still Requires Activation

```bash
# ❌ Wrong - venv exists but not activated
uv venv
python main.py  # Uses system Python!

# ✅ Correct
uv venv
source .venv/bin/activate
python main.py  # Uses venv Python
```

### 2. uv Commands Work Without Activation (Sometimes)

```bash
# uv can auto-detect .venv for these commands:
uv pip install fastapi  # Works even without activation!
uv pip list            # Shows .venv packages

# But Python still needs activation:
python -c "import fastapi"  # Fails if not activated!

# Solution: Always activate for consistency
```

### 3. Multiple Python Versions

```bash
# Check available Python versions
uv python list

# Install specific Python version
uv python install 3.12

# Create venv with specific version
uv venv --python 3.12
uv venv --python 3.11.5

# Pin Python version for project
uv python pin 3.12
```

### 4. uv.lock vs requirements.txt

```bash
# With uv project management (uv add)
# Creates uv.lock (exact versions + hashes)
uv add fastapi
uv sync  # Install from uv.lock

# Traditional workflow (uv pip install)
uv pip install fastapi
uv pip freeze > requirements.txt
uv pip install -r requirements.txt
```

### 5. Running Scripts Without Activation

```bash
# Run script with venv Python (no activation needed)
.venv/bin/python main.py  # Linux/macOS
.venv\Scripts\python main.py  # Windows

# Or use uv run (if uv project)
uv run python main.py
uv run uvicorn main:app
```

### 6. Global Tool Installation

```bash
# Install tools globally (outside venv)
uv tool install ruff
uv tool install black

# List global tools
uv tool list

# Run global tool
uvx ruff check .
```

## Common Workflows

### Workflow 1: Traditional (requirements.txt)

```bash
# Setup
mkdir myproject && cd myproject
uv venv
source .venv/bin/activate

# Install packages
uv pip install "fastapi[standard]"

# Save dependencies
uv pip freeze > requirements.txt

# Later: recreate environment
uv venv
source .venv/bin/activate
uv pip install -r requirements.txt
```

### Workflow 2: uv Project Management

```bash
# Setup
mkdir myproject && cd myproject
uv init  # Creates pyproject.toml

# Add dependencies
uv add "fastapi[standard]"
uv add --dev pytest

# This creates:
# - pyproject.toml (dependencies)
# - uv.lock (exact versions)
# - .venv/ (auto-created)

# Later: recreate environment
uv sync  # Reads uv.lock, creates .venv

# Run without activation
uv run python main.py
uv run uvicorn main:app
```

### Workflow 3: Clone Existing Project

```bash
# Clone project
git clone <repo>
cd project

# If using requirements.txt
uv venv
source .venv/bin/activate
uv pip install -r requirements.txt

# If using pyproject.toml + uv.lock
uv sync
```

## Quick Troubleshooting

```bash
# Problem: "Module not found" after installing
# Solution: Check if venv is activated
which python  # Should show .venv/bin/python

# Problem: Wrong Python version
# Solution: Recreate venv with specific version
rm -rf .venv
uv venv --python 3.12

# Problem: Packages installed in wrong venv
# Solution: Deactivate, activate correct one
deactivate
cd /correct/project
source .venv/bin/activate

# Problem: uv command not found
# Solution: Install uv
curl -LsSf https://astral.sh/uv/install.sh | sh  # Linux/macOS
# Or: pip install uv
```

## .gitignore Setup

```bash
# Manual method
echo "*" > .venv/.gitignore

# Or add to project .gitignore
cat >> .gitignore << EOF
.venv/
*.pyc
__pycache__/
*.egg-info/
EOF
```

**uv venv automatically creates `.venv/.gitignore`** ✅

## Environment Comparison

| Aspect | Global | Virtual Env |
|--------|--------|-------------|
| Location | System Python | `.venv/` in project |
| Packages | Shared across all projects | Isolated per project |
| Conflicts | ⚠️ Version conflicts | ✅ No conflicts |
| Activation | Always active | Must activate per terminal session |
| Use case | System tools | Application development |

## Python in Jupyter (This Notebook)

This notebook uses the Python kernel from your Jupyter installation. To ensure you're using your project's venv:

```bash
# Activate venv first
source .venv/bin/activate

# Install ipykernel in the venv
uv pip install ipykernel

# Register the venv as a Jupyter kernel
python -m ipykernel install --user --name=myproject

# Start Jupyter
jupyter notebook

# Select kernel: Kernel > Change kernel > myproject
```

## Command Cheatsheet

```bash
# Create venv
uv venv                    # Default: .venv/
uv venv --python 3.12      # Specific version

# Activate
source .venv/bin/activate  # Linux/macOS
.venv\Scripts\Activate.ps1 # Windows PS

# Deactivate
deactivate

# Install packages (traditional)
uv pip install package
uv pip install -r requirements.txt
uv pip freeze > requirements.txt

# Install packages (uv project)
uv add package
uv add --dev package
uv sync
uv remove package

# Check environment
which python               # Show Python path
uv pip list               # List packages
uv pip show package       # Package details

# Run without activation (uv project)
uv run python script.py
uv run uvicorn main:app
```

## Summary

**Core Pattern:**
1. Create venv: `uv venv`
2. Activate: `source .venv/bin/activate`
3. Install: `uv pip install` or `uv add`
4. Work on project
5. Deactivate when switching projects

**Key Points:**
- One venv per project
- Activation prepends `.venv/bin` to PATH
- Must activate in each new terminal session
- uv can auto-detect venv but activation still recommended
- Use `uv add` for full project management, `uv pip install` for simple projects

---

## Additional Resources

- [FastAPI Virtual Environments Docs](https://fastapi.tiangolo.com/virtual-environments/)
- [uv Documentation](https://docs.astral.sh/uv/)
- [Python venv Documentation](https://docs.python.org/3/library/venv.html)